# Trabajo práctico 1 : Analisis exploratorio del dataset Properatti

Participantes:

## Identificar el problema

Objetivos del trabajo: 

● General: Efectuar una limpieza del dataset provisto. 

● Especificos:
 - Realizar un análisis descriptivo de las principales variables.
 - Crear nuevas columnas a partir de las características dadas que puedan tener valor


## Adquirir los datos

In [ ]:
#librerias utilizadas para la adquisicion de los datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

TODO: otros recursos utilizados
- archivo de id_geonames: ar_copy.csv
- archivo de barrios Argentina: barrios.csv

In [ ]:
#abrir el dataset
data = pd.read_csv("./data/properatti.csv", index_col=0)

## Parsear los datos

### Exploración del dataset total de Properatti 

#### Describir las columnas del dataset:

In [ ]:
data.dtypes

In [ ]:
data.sample(5)

In [ ]:
# ver si existe algún dato cont todas sus columnas duplicadas
data.duplicated().any()

In [ ]:
data.dtypes

#### Análisis de datos faltantes

In [ ]:
print(data.isna().sum())

In [ ]:
data.isna().sum()/data.shape[0] *100

In [ ]:
missing_data = data.isna().sum(axis=0)
missing_data_df = pd.DataFrame(missing_data, columns=['count'])
missing_data_df['perc'] = (missing_data_df / data.shape[0]).round(2)*100
missing_data_df.sort_values(ascending=False, by='count')

#### Dispersión de datos

In [ ]:
data_dispersion = data.apply(lambda x: x.unique().size)
data_dispersion_df = pd.DataFrame(data_dispersion, columns=['count'])
data_dispersion_df["perc"] = (data_dispersion / data.shape[0]).round(2)*100
data_dispersion_df.sort_values(ascending=True, by='count')

In [ ]:
for col in data.columns:
    if(data[col].nunique() < 100):
        print(col)
        print(data[col].unique())
        print()

#### Analisis de correlacion entre columnas

### Exploración del dataset dividido en grupos de Properatti 

Para los siguientes pasos trabajaremos con el dataset dividido en tres grandes grupos:
- Localización: que contiene las columnas relacionadas con la ubicación del inmueble y a su vez este dividido en dos subgrupos:
  - Ubicación: que contiene las columnas place_name, state_name, country_name y place_with_parent_names, es decir columnas con la localización descriptiva del inmueble.
  - Geolocalización: que contiene las columnas geonames_id, lat y lon, es decir columnas con la localización geográfica del inmueble.
- Precio: que contiene las columnas relacionadas a los precios en distintas variantes. 
- Superficie: que contiene las columnas relacionadas con la superficie del inmueble

In [ ]:
# Definimos 4 grupos de columas para poder trabajar con ellas de forma mas sencilla
places = ['place_name','place_with_parent_names', 'country_name','state_name']
geolocation = ['geonames_id', 'lat-lon','lat','lon']
price = ['price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2']
surface = ['surface_total_in_m2','surface_covered_in_m2']

##### Ubicación

##### Geolocalización

##### Precio

##### Superficie

### Verificar la calidad de los datos:


## Minar los datos

## Refinar los datos

## Exportar el nuevo dataset 